In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
df=pd.read_excel("istanbul.xlsx",skiprows=1)
df.head()

C:\Users\K.G.LAKSSAN\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,date,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
0,2009-01-05,0.035754,0.038376,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524
1,2009-01-06,0.025426,0.031813,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773
2,2009-01-07,-0.028862,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015
3,2009-01-08,-0.062208,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424
4,2009-01-09,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802


In [5]:
df['date'] = df['date'].astype('int64') // 10**9
df.head()

,date,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
0,1231113600,0.035754,0.038376,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524
1,1231200000,0.025426,0.031813,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773
2,1231286400,-0.028862,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015
3,1231372800,-0.062208,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424
4,1231459200,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802


In [9]:
scalerx=MinMaxScaler()
scalery=MinMaxScaler()
xscaled=scalerx.fit_transform(df.iloc[:,:-1])
yscaled=scalery.fit_transform(df.iloc[:,-1].values.reshape(-1,1))

In [11]:
def create_seq(x,y,length):
    xseq=[]
    yseq=[]
    for i in range(len(x)-length):
        xseq.append(x[i:i+length])
        yseq.append(y[i+length])
    return np.array(xseq),np.array(yseq)

In [13]:
sequence_length=7
train_size=int(len(xscaled)*0.8)

In [17]:
xtrain_data=xscaled[:train_size]
xtest_data=xscaled[train_size-sequence_length:]
ytrain_data=yscaled[:train_size]
ytest_data=yscaled[train_size-sequence_length:]

In [21]:
xtrain,ytrain=create_seq(xtrain_data,ytrain_data,sequence_length)
xtest,ytest=create_seq(xtest_data,ytest_data,sequence_length)

In [27]:
model=Sequential([
    LSTM(64,return_sequences=True,input_shape=(sequence_length,xtrain.shape[2])),
    Dropout(0.1),
    LSTM(64,return_sequences=True),
    Dropout(0.1),
    LSTM(64),
    Dropout(0.1),
    Dense(1)
])

C:\Users\K.G.LAKSSAN\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [31]:
history=model.fit(xtrain,ytrain,epochs=40,batch_size=16,validation_split=0.2)

Epoch 1/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.1065 - val_loss: 0.0167
Epoch 2/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0218 - val_loss: 0.0143
Epoch 3/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0199 - val_loss: 0.0156
Epoch 4/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0190 - val_loss: 0.0147
Epoch 5/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0173 - val_loss: 0.0138
Epoch 6/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0161 - val_loss: 0.0153
Epoch 7/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0229 - val_loss: 0.0137
Epoch 8/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0200 - val_loss: 0.0136
Epoch 9/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0181 - val_loss: 0.0141
Epoch 10/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0223 - val_loss: 0.0155
Epoch 11/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0222 - val_loss: 0.0143
Epoch 12/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0161 - val_

In [33]:
train_predictions=model.predict(xtrain)
test_predictions=model.predict(xtest)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [35]:
train_predictions=scalery.inverse_transform(train_predictions)
ytrain_inv=scalery.inverse_transform(ytrain)
test_predictions=scalery.inverse_transform(test_predictions)
ytest_inv=scalery.inverse_transform(ytest)

In [43]:
import math
def calculate_metrics(actual, predicted):
    metrics = {
        'RMSE': math.sqrt(mean_squared_error(actual, predicted)),
        'MAE': mean_absolute_error(actual, predicted),
        'R²': r2_score(actual, predicted),
        'MAPE': np.mean(np.abs((actual - predicted) / (actual + 1e-5))) * 100
    }
    return metrics
    
train_metrics = calculate_metrics(ytrain_inv, train_predictions)
test_metrics = calculate_metrics(ytest_inv, test_predictions)

print("Training Metrics:", train_metrics)
print("Testing Metrics:", test_metrics)

Training Metrics: {'RMSE': 0.011136300982060771, 'MAE': 0.008287340770403715, 'R²': 0.00521999185646449, 'MAPE': 149.28284235085252}
Testing Metrics: {'RMSE': 0.006821336035245286, 'MAE': 0.005516716096050645, 'R²': -0.22566712636863917, 'MAPE': 216.84179232448156}
